# Будем парсить yandex market на предмет информации по детской литературе

К сожалению Yandex маркет определяет, что его хочет распарить бот через (requests) и блокирует, после пары вечеров мучений выбрал selenium для обхода проблемы. Также selenium должен помочь решить проблему с ajax скриптами, которые подгружают часть информации во время серфинга конкретной страницы. 

In [2613]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import re

Добавляю опции для дополнительной защиты от отслеживания ботов и исключения сохранения cookie

In [2614]:
option = webdriver.ChromeOptions()
option.add_argument("--disable-infobars")
option.add_argument("--window-size=1000,1200")
option.add_argument("--incognito")
option.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=option)

URL базовой страницы для заданной категории в нашем случае “Детские книги”

In [2615]:
base_url = 'https://market.yandex.ru/catalog--novinki-knig/43762671/list?rs=eJwdUCurQkEY3KNJ_AWCwpoNos0XHE3CRRAxXuEk8T9YDj7AICabZbMoaLD4wHODiE0QwwXBgyI2g3AV292ZMgwz3zfz7cYb3qKxMYSoZjRa5ZRG55bWaF4WGu1jUqPbg-I84YoRdNMm_4EuOkAVXGK-wK07FHcCbs3pRuCqPpUjuPwCihJ6zTBzWnDllvMzoP2dQM6H90yB1h963foafAdXxrhbYeMQmWaemW_qPnDZBKo0bzDoRtGlunxvm10v9h740jHdAXv3_IER-S8nT9iyrrxHUg-toOfY6GG-n5OKaQEmn_k_Nmey4M6DLbXUP1YoiHA%2C&allowCollapsing=1&local-offers-first=0&page='

Функция отвечает за скролл в конец страницы и подгруздку всех необходимых тэгов

In [2616]:
def scroll_down_page(dr):
    SCROLL_PAUSE_TIME = 2 
    last_height = 0
    while True:
        #Случайный интервал времени
        t_rnd = random.randint(0, 10)
        #Случайное количкство пикселей на скрол
        y_rnd = random.randint(200, 250)
        driver.execute_script("window.scrollTo(0, window.scrollY + " + str(y_rnd) +")")
        time.sleep(t_rnd / 10)
        height = int(driver.execute_script("return window.pageYOffset + window.innerHeight"))
        if height == last_height:
            break
        last_height = height


Функция отвечает за возврат всех ссылок на дочерние страницы по продуктам с них мы будем забирать информацию

In [2617]:
def return_hrefs_on_page(wd, base_url, pg):
    page_url = base_url + str(pg)
    list = []
    driver.get(page_url)
    scroll_down_page(driver)
    products = driver.find_elements(By.CLASS_NAME, "_2im8-._2S9MU._2jRxX")
    for item in products:
        a_href = item.find_element(By.TAG_NAME, "a")
        list.append(a_href.get_attribute('href'))
    return list

ИницииФункция инициирует подгруздку информации, которая получается после нажатия кнопки и вызова Ajax. На дефолтной странице этой информации по началу нет

In [2618]:
def ajax_init(wd):
    try:
        action = webdriver.ActionChains(wd)
        cursor = wd.find_element(By.CLASS_NAME,"_2AMPZ._2Fly2._1ghok._165o6")
        action.move_to_element(cursor)
        action.perform()
        time.sleep(1)
        return True
    except:
        return False


Далее набор функций парусящих необходимую для нас информацию

In [2619]:
def return_rating(wd):
    rating_elms = wd.find_elements(By.CLASS_NAME, "_3RoU0")
    try:
        for item in rating_elms:
             if 'Рейтинг товара' in item.get_attribute("innerText"):
                rating = re.search(r'Рейтинг товара:[ ](\d\.\d)', item.get_attribute('innerHTML'))
                return rating[1] if rating else 0.0
    except:
            return 0.0
    

In [2620]:
def return_reviews(wd):   
    try:
        el = wd.find_element(By.CLASS_NAME, "_oz_z")
        return int(el.text.split('/')[1])
    except:
        return 0
    

In [2621]:
def return_green_product_price(wd):
   try:
      price_element = wd.find_element(By.CLASS_NAME, "_3BWiE")
      price = re.search(r'Цена с картой Яндекс Пэй:</span>(\d+)', price_element.get_attribute('innerHTML'))
      return int(price[1]) if price else 0
   except:
      return 0


In [2622]:
def return_title_book(wd):
    try:
        title_h1 = wd.find_element(By.TAG_NAME, "h1")
        return title_h1.text
    except:
        return 'Нет'

In [2623]:
def return_description(wd):
    try:
       descr_elements = wd.find_element(By.CLASS_NAME,"_2Oxr4")
       return descr_elements.text.replace("\n", "")
    except:
        return 'Нет'

In [2624]:
def return_properties(wd):
    try:
        dict = {}
        short_description = wd.find_elements(By.CLASS_NAME,"_198Aj.cXkP_._3wss4._1XOOj")
        for item in short_description:
            key = re.search(r'<div class=\"_1oG4-\"><span class=\"_2NZVF _15CQ5 _32rOe _25vcL\">([\w ]+)<.+', item.get_attribute('innerHTML'))
            value = re.search(r'<span class=\".+\">(.+)</span>', item.get_attribute('innerHTML'))
            dict[key[1]] = value[1]
        return dict
    except:
        return {}
        

In [2625]:
final_df = pd.DataFrame(columns=["Рейтинг",  "Отзывы", "Цена", "Заголовок", "Описание", "URL"])
final_df = final_df.astype(dtype= {"Рейтинг":"float", "Отзывы": "int16", "Цена": "int32", "Заголовок":"string", "Описание": "string", "URL": "string"})

Корневая функция вызывающая все остальные и собирающая информацию в единый словарь

In [2626]:
def getProductData(wd, url):
    try:
        ajax_init(wd)
        return_green_product_price(wd)
        price = return_green_product_price(wd)
        rating = return_rating(wd)
        reviews = return_reviews(wd)
        title = return_title_book(wd)
        description = return_description(wd)
        pr = return_properties(wd)
        data_row = { "Рейтинг":rating, "Отзывы":reviews, "Цена": price, "Заголовок":title, "Описание": description, "URL": url }
        return pr | data_row
    except:
        print("Проблема парсинга", url, " ",rating," ",reviews," ",price," ", title)
        
        return {}


Перебираем заданное количество страниц в нашей категории получаем всю информацию о товаре и сохраняем в датафрейм

In [2627]:
rating_list = []
for i in range(1, 3):
    for url in return_hrefs_on_page(driver, base_url, i)[1:7]:
        t_rnd = random.randint(0, 20)
        time.sleep(t_rnd / 10)
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(url)
        data_row = getProductData(driver, url)
        if data_row:
            final_df = pd.concat([final_df, pd.DataFrame.from_records([data_row])], ignore_index=True)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

<div class="cia-vs cia-cs" data-node-id="lxwf_88nw_1" data-zone-data="{&quot;props&quot;:{&quot;title&quot;:&quot;Еще может подойти&quot;},&quot;garsons&quot;:[{&quot;id&quot;:&quot;RecommendedSnippets&quot;,&quot;count&quot;:32,&quot;params&quot;:{&quot;productId&quot;:1921866858,&quot;experiment&quot;:&quot;CompetitiveAnalogs_MarketModelCard_FrontSnippet&quot;,&quot;cpc&quot;:&quot;k-13FNxCag-loKh52rs8Cy4av9E1flUMKcU4xZ4936KmEudvM-DnB6HR5ZQuj_uHbZfwwl-d3BR-1ey9vS2woFeoLUJb9CW_iKD6W5oXYaiV0vfUyrRNr3t2PrUAyC7Nu9yptCZK3ICkpIDFgnf5Z9KTwBulgqVhk3AZElF9u2a7roAcaW8H349fAFYvbJP4q2NJrhQAbAzw0t5mNRlDFesl9WyfocfsvfsYmI7tXzIlcJ44-eAFhw6nR3RBpq4mR7ZOEdzWpUfHxmQohAMSWw,,&quot;}}],&quot;isLight&quot;:true,&quot;blockType&quot;:&quot;scrollbox&quot;}" data-zone-name="ScrollBox" data-baobab-name="block"><div class="_1p2EP cXkP_ gCOkS" data-auto="scrollbox"><div data-apiary-widget-name="@light/ScrollBoxHeader" data-apiary-widget-id="/content/page/fancyPage/productPageLegacy/reactProductSummary/product

In [2628]:
final_df.loc[[1]]

,Рейтинг,Отзывы,Цена,Заголовок,Описание,URL,ISBN,Направление,Автор,Серия,...,Датированность,Размер,Обложка,Крепление,Особенности,Дополнительная информация,Издательство,Минимальный возраст,Тип графической книги,Подарочное издание
1,0.0,3746,238,Книга Русь изначальная,"Роман Валентина Иванова «Русь изначальная» принадлежит к условной исторической трилогии, посвященной времени становления Русского государства — «эпохе дальней, о которой еще никто не писал». С момента первой публикации в 1961 году этот монументальный эпос заслуженно пользуется любовью читателей, выдержал множество переизданий, а в 1985 году сюжет романа лег в основу одноименного фильма. VI век нашей эры. Племена восточных славян разрозненны, но они говорят на одном языке и живут сходными традициями. Вынужденные обороняться против разбойничьих набегов иноверцев-кочевников, они начинают задумываться об объединении перед лицом общего врага… Детально проработанный образ славянского быта и традиций в сочетании с увлекательным сюжетом и выразительными портретами персонажей завоевали книгам Валентина Иванова почетное место среди лучших образцов жанра исторического романа.",https://market.yandex.ru/product--rus-iznachalnaia/1930176335?nid=20598910&show-uid=16982172465598798230316003&context=search&uniqueId=5415678&sku=102386071846&cpc=k-13FNxCag-yqc4hFRFxjCu3CdlZ8L-rBWmiVyZlwch0FkAauI0K019pOYGId-teDdsrjYI_tinPSgn2gH2Dja23U4XWQ54ehQVOpvtsiYttsXvLzAULCdBE9e11Imb8PDohoHE1WUf_RVMn0r5iZBxCEXd9FvaFYL7BatlLHxzb6rd9iCGPZgpALLhbgu4JJ-V_FTbaepg%2C&do-waremd5=Yjw9lOZIr_qV2fwi20vA6g&rs=eJwdUCurQkEY3KNJ_AWCwpoNos0XHE3CRRAxXuEk8T9YDj7AICabZbMoaLD4wHODiE0QwwXBgyI2g3AV292ZMgwz3zfz7cYb3qKxMYSoZjRa5ZRG55bWaF4WGu1jUqPbg-I84YoRdNMm_4EuOkAVXGK-wK07FHcCbs3pRuCqPpUjuPwCihJ6zTBzWnDllvMzoP2dQM6H90yB1h963foafAdXxrhbYeMQmWaemW_qPnDZBKo0bzDoRtGlunxvm10v9h740jHdAXv3_IER-S8nT9iyrrxHUg-toOfY6GG-n5OKaQEmn_k_Nmey4M6DLbXUP1YoiHA%2C,9785389238381,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2629]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Рейтинг                    12 non-null     float64
 1   Отзывы                     12 non-null     int64  
 2   Цена                       12 non-null     int64  
 3   Заголовок                  12 non-null     object 
 4   Описание                   12 non-null     object 
 5   URL                        12 non-null     object 
 6   ISBN                       10 non-null     object 
 7   Направление                3 non-null      object 
 8   Автор                      7 non-null      object 
 9   Серия                      7 non-null      object 
 10  Язык                       6 non-null      object 
 11  Год издания                8 non-null      object 
 12  Тираж                      5 non-null      object 
 13  Количество страниц         7 non-null      object 
 